In [34]:
import cv2
import numpy as np
from pathlib import Path

class PlantHealthMonitor:
    def __init__(self):
        # Define color ranges for health indicators (in HSV)
        self.lower_healthy = np.array([35, 40, 40])
        self.upper_healthy = np.array([85, 255, 255])
        
        self.lower_yellow = np.array([15, 50, 50])
        self.upper_yellow = np.array([35, 255, 255])
        
        self.lower_brown = np.array([0, 50, 20])
        self.upper_brown = np.array([15, 255, 200])
    
    def load_image(self, path):
        """Load image from file"""
        image = cv2.imread(path)
        if image is None:
            raise ValueError(f"Could not load image from {path}")
        return image
    
    def isolate_leaf(self, image):
        """Separate leaf from background using color thresholds"""
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        
        lower_plant = np.array([15, 30, 30])
        upper_plant = np.array([90, 255, 255])
        plant_mask = cv2.inRange(hsv, lower_plant, upper_plant)
        
        kernel = np.ones((5, 5), np.uint8)
        plant_mask = cv2.morphologyEx(plant_mask, cv2.MORPH_CLOSE, kernel)
        plant_mask = cv2.morphologyEx(plant_mask, cv2.MORPH_OPEN, kernel)
        
        contours, _ = cv2.findContours(plant_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if not contours:
            return None, None
        
        largest_contour = max(contours, key=cv2.contourArea)
        
        leaf_mask = np.zeros_like(plant_mask)
        cv2.drawContours(leaf_mask, [largest_contour], 0, 255, -1)
        
        leaf_only = cv2.bitwise_and(image, image, mask=leaf_mask)
        
        return leaf_only, leaf_mask
    
    def analyze_color_health(self, image, leaf_mask):
        """Analyze color distribution to detect healthy vs. unhealthy areas"""
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        
        healthy_mask = cv2.inRange(hsv, self.lower_healthy, self.upper_healthy)
        yellow_mask = cv2.inRange(hsv, self.lower_yellow, self.upper_yellow)
        brown_mask = cv2.inRange(hsv, self.lower_brown, self.upper_brown)
        
        healthy_area = cv2.bitwise_and(healthy_mask, leaf_mask)
        yellow_area = cv2.bitwise_and(yellow_mask, leaf_mask)
        brown_area = cv2.bitwise_and(brown_mask, leaf_mask)
        
        leaf_area = cv2.countNonZero(leaf_mask)
        if leaf_area == 0:
            return 0, 0, 0
        
        healthy_percentage = (cv2.countNonZero(healthy_area) / leaf_area) * 100
        yellow_percentage = (cv2.countNonZero(yellow_area) / leaf_area) * 100
        brown_percentage = (cv2.countNonZero(brown_area) / leaf_area) * 100
        
        return healthy_percentage, yellow_percentage, brown_percentage
    
    def detect_leaf_damage(self, image, leaf_mask):
        """Detect spots or lesions on the leaf"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        leaf_gray = cv2.bitwise_and(gray, gray, mask=leaf_mask)
        
        blurred = cv2.GaussianBlur(leaf_gray, (5, 5), 0)
        
        _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        kernel = np.ones((3, 3), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
        
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        min_spot_area = 25
        spots = [cnt for cnt in contours if cv2.contourArea(cnt) > min_spot_area]
        
        spots_area = sum(cv2.contourArea(cnt) for cnt in spots)
        leaf_area = cv2.countNonZero(leaf_mask)
        if leaf_area == 0:
            return 0, 0
        
        damage_percentage = (spots_area / leaf_area) * 100
        spots_count = len(spots)
        
        return damage_percentage, spots_count
    
    def analyze_texture(self, image, leaf_mask):
        """Analyze leaf texture for signs of wilting or other abnormalities"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        leaf_gray = cv2.bitwise_and(gray, gray, mask=leaf_mask)
        
        laplacian = cv2.Laplacian(leaf_gray, cv2.CV_64F)
        
        texture_std = np.std(laplacian)
        
        if texture_std > 20:
            texture_status = "High (Possible Wilting)"
        elif texture_std > 10:
            texture_status = "Medium"
        else:
            texture_status = "Low (Smooth/Healthy Texture)"
        
        return texture_std, texture_status
    
    def analyze_health(self, image_path):
        """Analyze plant health but only return requested metrics"""
        # Load image
        image = self.load_image(image_path)
        
        # Isolate leaf from background
        leaf_image, leaf_mask = self.isolate_leaf(image)
        
        if leaf_image is None:
            return {"error": "No leaf detected in image"}
        
        # Analyze color health
        healthy_pct, yellow_pct, brown_pct = self.analyze_color_health(image, leaf_mask)
        
        # Detect leaf damage
        damage_pct, spots_count = self.detect_leaf_damage(image, leaf_mask)
        
        # Analyze texture
        texture_std, texture_status = self.analyze_texture(image, leaf_mask)
        
        # Calculate overall health score
        health_score = healthy_pct - (yellow_pct + brown_pct + damage_pct)
        
        # Determine overall health status
        if texture_std > 20:
            status = "High (Possible Wilting)"
        elif texture_std > 10:
            status = "Medium"
        else:
            status = "Low (Smooth/Healthy Texture)"

        
        # Compile only the requested information
        results = {
            'status': status,
            'texture_variation': texture_std,
            'texture_status': texture_status
        }
        
        return results

def main(image_path=None):
    if image_path is None:
        image_path = input("Enter analyze image from file: ")
        
    monitor = PlantHealthMonitor()
        
    try:
        results = monitor.analyze_health(image_path)
        
        # Print only the requested results
        print("\nPlant Health Analysis:")
        print(f"status: {results['status']}")
        print(f"texture_variation: {results['texture_variation']:.2f}")
        print(f"texture_status: {results['texture_status']}")
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    
    image_paths = []
    
    input_path = Path('images/leaf count/new folder')
    for image_file in input_path.rglob('*.jpg'):
        image_paths.append(image_file)
    
    analyzed_images = []
    
    # Sort images to ensure consistent order for time series
    for image_path in sorted(image_paths):
        main(image_path)



Plant Health Analysis:
status: Low (Smooth/Healthy Texture)
texture_variation: 0.78
texture_status: Low (Smooth/Healthy Texture)

Plant Health Analysis:
status: Low (Smooth/Healthy Texture)
texture_variation: 2.22
texture_status: Low (Smooth/Healthy Texture)

Plant Health Analysis:
status: Low (Smooth/Healthy Texture)
texture_variation: 1.85
texture_status: Low (Smooth/Healthy Texture)

Plant Health Analysis:
status: Low (Smooth/Healthy Texture)
texture_variation: 6.53
texture_status: Low (Smooth/Healthy Texture)

Plant Health Analysis:
status: Low (Smooth/Healthy Texture)
texture_variation: 8.63
texture_status: Low (Smooth/Healthy Texture)

Plant Health Analysis:
status: Low (Smooth/Healthy Texture)
texture_variation: 8.71
texture_status: Low (Smooth/Healthy Texture)

Plant Health Analysis:
status: Low (Smooth/Healthy Texture)
texture_variation: 6.24
texture_status: Low (Smooth/Healthy Texture)

Plant Health Analysis:
status: Low (Smooth/Healthy Texture)
texture_variation: 5.85
textu